# AutoGen のトレースを Jaeger で永続化

AutoGenには、アプリケーションの実行に関する包括的な記録を収集するためのトレースと観測のサポートが組み込まれています。この機能は、デバッグ、パフォーマンス分析、そしてアプリケーションのフローを理解するのに役立ちます。

この機能はOpenTelemetryライブラリを活用しているため、OpenTelemetry と互換性のある任意のバックエンドを使用してトレースを収集および分析できます。

## 設定
まず、OpenTelemetry Pythonパッケージをインストールする必要があります。pipを使ってインストールできます。


In [ ]:
!pip install opentelemetry-sdk opentelemetry-exporter-otlp-proto-grpc

## AutoGen でトレースを設定

1. OpenTelemetry トレーサープロバイダーを構成する
1. バックエンドにトレースを送信するためのエクスポーターを設定する
1. トレーサープロバイダを AutoGen ランタイムに接続する

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()


## シンプルなトレース

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient
import os

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4.1-mini",
    model="gpt-4.1-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMessageTermination  

# Create the final reviewer agent
writer_agent = AssistantAgent(
    "writer_agent",
    model_client=model_client,
    system_message="あなたはプロの小説家です。魅力的な文章を完結なタッチで書くことができます。",
)

In [ ]:
stream = writer_agent.run_stream(task="SEが異世界に転生して無双する異世界転生系短編小説を書いてください。")
await Console(stream)

## 独自のスパンを作成
複雑なアプリケーションを開発するときに、ビジネス ロジックとモデルを組み合わせたコードのセクションをキャプチャすると便利な場合があります。 OpenTelemetry では、スパンの概念を使用して、関心のあるセクションをキャプチャします。 独自のスパンの生成を開始するには、現在の トレーサー オブジェクトのインスタンスを取得します。

In [ ]:
from typing import List

@tracer.start_as_current_span("write_and_critic_short_story")
async def write_and_critic_short_story():
    current_span = trace.get_current_span()
    # Set attributes for the current span
    current_span.set_attribute("operation.critic_count", 3)


    # Create the final reviewer agent
    writer_agent = AssistantAgent(
        "writer_agent",
        model_client=model_client,
        system_message="あなたはプロの小説家です。魅力的な文章を完結なタッチで書くことができます。",
    )


    # Create the final reviewer agent
    critic_agent1 = AssistantAgent(
        "critic_agent1",
        model_client=model_client,
        system_message="作成された文章を批判的な観点から評価し、改善点を提案すること。",
    )

    # Create the final reviewer agent
    critic_agent2 = AssistantAgent(
        "critic_agent2",
        model_client=model_client,
        system_message="あなたはリスクアドバイザーです。作成された文章のアイデア被りやSMS等で炎上しないかどうかを評価し、改善点を提案すること。",
    )

    # Create the final reviewer agent
    critic_agent3 = AssistantAgent(
        "critic_agent3",
        model_client=model_client,
        system_message="あなたは法学のスペシャリストです。作成された文章を法的観点から評価し、改善点を提案すること。",
    )

    # add agents to array
    critic_agents: List[AssistantAgent] = [
        writer_agent,
        critic_agent1,
        critic_agent2,
        critic_agent3,
    ]

    termination_condition = TextMessageTermination("critic_agent3") #3人目が発言したら終了
    critic_team = RoundRobinGroupChat(critic_agents, termination_condition=termination_condition)
            
    stream = critic_team.run_stream(task="SEが異世界に転生して無双する異世界転生系短編小説を書いてください。")
    await Console(stream)

    return

In [ ]:
await write_and_critic_short_story()